## Softmax classifier approach

This approach assumes that quantifiers are learned as a group and that essentially each q quantifier example is a negative example for quantifiers q' with q not equal to q. The interaction between quantifer 

## Global Imports

In [1]:
import numpy as np
import pandas as pd

# %load_ext autoreload
# %autoreload 2
from quants import *

### keras imports

In [2]:
# keras imports

import tensorflow as tf

print("TensorFlow version: ", tf.__version__)

import keras

from keras.models import Sequential, Model
from keras.layers import SimpleRNN, LSTM, Embedding, Dense, Conv1D, Conv2D, MaxPooling1D, UpSampling1D, Input, Bidirectional, RepeatVector, TimeDistributed, Dropout, LeakyReLU, Flatten
from keras.utils import plot_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

TensorFlow version:  2.2.0


Using TensorFlow backend.


In [3]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU')]

In [4]:
# from functools import partial, update_wrapper

# def wrapped_partial(func, *args, **kwargs):
#         partial_func = partial(func, *args, **kwargs)
#         update_wrapper(partial_func, func)
#         return partial_func

## Classifier models

In [5]:
def CNNClassifier():
    model= Sequential()
    model.add(Embedding(len(symbols), 3, input_length=scene_len))
#     model.add(Conv2D(filters = 1, kernel_size=(1,3), strides=1, activation='sigmoid',padding='valid', input_shape=(scene_len, 3,1)))
    model.add(Flatten())
    model.add(Dropout(0.25))
    model.add(Dense(len(natural_quantifiers), activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### learn

In [11]:
num_epochs = 500
batch_size = 20

def learn(scenes, labels):
    model = CNNClassifier()
    plot_model(model, to_file='classifier_model.png', show_shapes=True)
    model.fit(scenes, labels, batch_size=batch_size, epochs=num_epochs, verbose=1)
    return model

### test

In [12]:
def test_quantifer(model, quantifier):
    test_size = 100

    test_scenes = generate_random_scenes(scenes_num=test_size)
    test_labels = [quantifier.quantify(test_scene) for test_scene in test_scenes]
    test_results = [model.evaluate(test_scene.reshape(1, -1, 1),
                                   verbose=0) for test_scene in test_scenes]
    thresh = np.mean(test_results)
    test_thresh = [test_result < thresh for test_result in test_results]
    print(np.sum(np.array(test_labels) == np.array(test_thresh)))

In [13]:
natural_quantifiers = [Both(), No(), Every(), Few(), Some(), Most()]

In [14]:
unnatural_quantifiers = [MinMax(2, 10), MinMax(3, 6), Or([MinMax(2, 5), MinMax(10, 20)])]

In [15]:
# with tf.device("/cpu:0"):
with tf.device("/gpu:0"):
    # generate scenes and labels
    scenes = np.vstack([generate_quantified_scenes(q, scene_num) for q in natural_quantifiers])
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit([q.name() for q in natural_quantifiers])
    encoded_Y = encoder.transform(np.concatenate([[q.name()] * scene_num
                                                  for q in natural_quantifiers]).reshape(-1, 1))
    # convert integers to dummy variables (i.e. one hot encoded)
    labels = np_utils.to_categorical(encoded_Y)

    # fit model
#     model = learn(Quantifier.one_hot(scenes).reshape(6000,100,3,1), labels)
    model = learn(scenes, labels)

/home/doron/git/research/RESEARCH/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/doron/git/research/RESEARCH/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/500
6000/6000 [==============================] - 0s 83us/step - loss: 1.1383 - accuracy: 0.5263
Epoch 2/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.8710 - accuracy: 0.6352
Epoch 3/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.7912 - accuracy: 0.6698
Epoch 4/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.7351 - accuracy: 0.6998
Epoch 5/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.7029 - accuracy: 0.7108
Epoch 6/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.6759 - accuracy: 0.7287
Epoch 7/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.6516 - accuracy: 0.7347
Epoch 8/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.6398 - accuracy: 0.7373
Epoch 9/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.6299 - accuracy: 0.7442
Epoch 10/500
6000/6000 [==============================] - 0s 73u

6000/6000 [==============================] - 0s 75us/step - loss: 0.5270 - accuracy: 0.7742
Epoch 80/500
6000/6000 [==============================] - 0s 75us/step - loss: 0.5159 - accuracy: 0.7817
Epoch 81/500
6000/6000 [==============================] - 0s 75us/step - loss: 0.5247 - accuracy: 0.7772
Epoch 82/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.5200 - accuracy: 0.7792
Epoch 83/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.5175 - accuracy: 0.7867
Epoch 84/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.5158 - accuracy: 0.7785
Epoch 85/500
6000/6000 [==============================] - 0s 76us/step - loss: 0.5215 - accuracy: 0.7763
Epoch 86/500
6000/6000 [==============================] - 0s 78us/step - loss: 0.5196 - accuracy: 0.7805
Epoch 87/500
6000/6000 [==============================] - 0s 78us/step - loss: 0.5083 - accuracy: 0.7825
Epoch 88/500
6000/6000 [==============================] - 0s 78us/st

6000/6000 [==============================] - 0s 75us/step - loss: 0.4812 - accuracy: 0.7945
Epoch 157/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4781 - accuracy: 0.7992
Epoch 158/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4817 - accuracy: 0.7955
Epoch 159/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4752 - accuracy: 0.7995
Epoch 160/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4774 - accuracy: 0.7972
Epoch 161/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4773 - accuracy: 0.7982
Epoch 162/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4755 - accuracy: 0.8020
Epoch 163/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4811 - accuracy: 0.7978
Epoch 164/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4752 - accuracy: 0.7953
Epoch 165/500
6000/6000 [==============================] - 0

6000/6000 [==============================] - 0s 74us/step - loss: 0.4632 - accuracy: 0.8038
Epoch 234/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4529 - accuracy: 0.8053
Epoch 235/500
6000/6000 [==============================] - 0s 75us/step - loss: 0.4432 - accuracy: 0.8107
Epoch 236/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4461 - accuracy: 0.8113
Epoch 237/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4552 - accuracy: 0.8067
Epoch 238/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4537 - accuracy: 0.8065
Epoch 239/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4547 - accuracy: 0.8083
Epoch 240/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4553 - accuracy: 0.8102
Epoch 241/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4534 - accuracy: 0.8108
Epoch 242/500
6000/6000 [==============================] - 0

6000/6000 [==============================] - 0s 75us/step - loss: 0.4336 - accuracy: 0.8125
Epoch 311/500
6000/6000 [==============================] - 0s 75us/step - loss: 0.4391 - accuracy: 0.8103
Epoch 312/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4385 - accuracy: 0.8150
Epoch 313/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4379 - accuracy: 0.8120
Epoch 314/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4390 - accuracy: 0.8115
Epoch 315/500
6000/6000 [==============================] - 0s 75us/step - loss: 0.4320 - accuracy: 0.8180
Epoch 316/500
6000/6000 [==============================] - 0s 77us/step - loss: 0.4350 - accuracy: 0.8172
Epoch 317/500
6000/6000 [==============================] - 0s 78us/step - loss: 0.4441 - accuracy: 0.8140
Epoch 318/500
6000/6000 [==============================] - 0s 77us/step - loss: 0.4388 - accuracy: 0.8095
Epoch 319/500
6000/6000 [==============================] - 0

6000/6000 [==============================] - 0s 75us/step - loss: 0.4330 - accuracy: 0.8088
Epoch 388/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4390 - accuracy: 0.8087
Epoch 389/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4269 - accuracy: 0.8163
Epoch 390/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.4291 - accuracy: 0.8198
Epoch 391/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4355 - accuracy: 0.8145
Epoch 392/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4436 - accuracy: 0.8095
Epoch 393/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.4452 - accuracy: 0.8055
Epoch 394/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4385 - accuracy: 0.8153
Epoch 395/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4333 - accuracy: 0.8173
Epoch 396/500
6000/6000 [==============================] - 0

6000/6000 [==============================] - 0s 74us/step - loss: 0.4352 - accuracy: 0.8110
Epoch 465/500
6000/6000 [==============================] - 0s 73us/step - loss: 0.4308 - accuracy: 0.8165
Epoch 466/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4244 - accuracy: 0.8162
Epoch 467/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4328 - accuracy: 0.8200
Epoch 468/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4232 - accuracy: 0.8180
Epoch 469/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4332 - accuracy: 0.8195
Epoch 470/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4419 - accuracy: 0.8088
Epoch 471/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4394 - accuracy: 0.8093
Epoch 472/500
6000/6000 [==============================] - 0s 74us/step - loss: 0.4353 - accuracy: 0.8112
Epoch 473/500
6000/6000 [==============================] - 0

In [16]:
# with tf.device("/cpu:0"):
with tf.device("/gpu:0"):
    test_quantifer(model, natural_quantifiers[0])

TypeError: generate_random_scenes() got an unexpected keyword argument 'scenes_num'